# 🎉End-to-End-Natural Language Processing-4 

### Recap the salient concepts demonstrated in the previous kernels:
- Cleaning the text data free from the stop words, punctuations, numbers, special characters, emojis, email IDs, websites, etc
- Lemmating the text data to the root word.e.g., went -> go
- Stemming the words to avoid duplication of the words.e.g., raining -> rain
- Exploratory analysis of text data
- Word clouds and Ngram analysis
- Non-semantic Vectorization of words (One Hot, TF-IDF)
- Dimension reduction techniques (PCA,SVD,TSNE) and visualisation of Ndim vectors in 2D space
- Statistical and ensemble algorithms
- Semantic vectorization of words in Ndim space - Static word embeddings
- Word2Vec algorithm - CBOW and Skipgram - Word Embeddings in vector space
- Pre-trained word embeddings: GloVe, GoogleNews, fasttext 
- Neural Networks - Convolutional, Recurrent, and BiRNN with pre-trained embeddings

## In this kernel, I am exploring as well as building the following models:

1. Seq2Seq Model (Encoder-Decoder without Attention) - NMT
2. Dynamic embeddings (Pre-trained word embeddings on Bidirectional LSTMs): ELMo
3. Seq2Seq Models (Encoder-Decoder with Bahdanu, Luong Attention,etc)
4. Neural Machine Translation Code for Seq2Seq model with attention on English-Spanish data set
5. Transformer Architecture Explained

**As always, I hope you find this 'notebook' useful.**

[UPVOTES](https://www.kaggle.com/rizdelhi/quora-insincere-questions-part-4-bert) would be highly appreciated and great motivation.

In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)   
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords   
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# generic function to plot the train Vs validation loss/accuracy:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    plt.figure(figsize=(25,15))
    ## Accuracy
    plt.subplot(2,2,1)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.4f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.4f'))+')')

    plt.title('Training Accuracy Vs Validation Accuracy\n')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    ## Loss
    plt.subplot(2,2,2)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.4f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.4f'))+')'))
    
    plt.title('Training Loss Vs Validation Loss\n')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

### Train data

In [ ]:
# Load the input features

def load_train():
    pd.set_option('display.max_colwidth',None)
    train =pd.read_csv('../input/clean-quora-train-data/clean_lem_stemmed_train_data.csv') 
    train=train.dropna()
    return train

In [ ]:
train_df = load_train()
train_df.head()

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from tensorflow.keras import regularizers
from keras import layers
from tensorflow.keras.layers import Embedding, Bidirectional, GlobalMaxPool1D
from gensim.models import KeyedVectors
from tensorflow.keras import regularizers
from keras import layers
from tensorflow.keras.layers import Embedding
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense,Flatten,Conv2D,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical

In [ ]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer

X = train_df['question_text'] # input
y = train_df['target'].values # target /label

sentences_train,sentences_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=11)

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_val = tokenizer.texts_to_sequences(sentences_val)

# Adding 1 because of  reserved 0 index
vocab_size = len(tokenizer.word_index) + 1 # (in case of pre-trained embeddings it's +2)                         
maxlen = 131 # sentence length

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)


print("Padded and Tokenized Training Sequence".format(),X_train.shape)
print("Target Training Values Shape".format(),y_train.shape)
print("_____________________________________________")
print("Padded and Tokenized Validation Sequence".format(),X_val.shape)
print("Target Validatation Values Shape".format(),y_val.shape)

In [ ]:
num_tokens=len(tokenizer.word_index)+2
print("Number of Features/Tokens:",num_tokens)

In [ ]:
# delete unused varibles
del train_df
import gc
gc.collect()

## Sequence-to-Sequence (seq2seq) Model

Sequence-to-sequence (seq2seq) models (Sutskever et al., 2014, Cho et al., 2014) have enjoyed great success in a variety of tasks such as machine translation, speech recognition, and text summarization. 

> ### **Neural Machine Translation (NMT) which was the very first testbed for seq2seq models is wild success!**

Encoder-decoder is the standard modeling paradigm for sequence-to-sequence tasks. It can capture long-range dependencies in languages, e.g., gender agreements; syntax structures; etc., and produce much more fluent translations as demonstrated by Google Neural Machine Translation systems.


A natural choice for sequential data is the recurrent neural network (RNN), used by most NMT models. Usually an RNN is used for both the encoder and decoder. The RNN models, however, differ in terms of: (a) directionality – unidirectional or bidirectional; (b) depth – single- or multi-layer; and (c) type – often either a vanilla RNN, a Long Short-term Memory (LSTM), or a gated recurrent unit (GRU). 


[📖 READ the tutorial on NMT](https://github.com/tensorflow/nmt)

### Let's understand how Encoder - Decoder works?

> ### Encoder-decoder Framework 

![Neural Machine Translation - Source: Github ](https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/g3doc/img/encdec.jpg)

This framework consists of two components:

1. encoder - reads source sequence(inputs) to build a "thought" vector (context vector)

2. decoder - uses context vector from the encoder to generate the target sequence

sequence-to-sequence models need to estimate the conditional probability p(y|x) of a sequence y given a source x.That's why sequence-to-sequence tasks can be modeled as Conditional Language Models (CLM) - they operate similarly to LMs, but additionally receive source information x.

Seq2seq models are trained to predict probability distributions of the next token given previous context (source and previous target tokens). Intuitively, at each step we maximize the probability a model assigns to the correct token.

![Lena Voita's Blog](https://lena-voita.github.io/resources/lectures/seq2seq/general/enc_dec_prob_idea.mp4)

![Encoder Decoder- Lena Voita's Blog](https://lena-voita.github.io/resources/lectures/seq2seq/general/enc_dec_linear_out-min.png)

### What happens under the hood

RNN takes two inputs at each time step: an input (in the case of the encoder, one word from the input sentence), and a hidden state. The word, however, needs to be represented by a vector. To transform a word into a vector, we turn to the class of methods called “word embedding” algorithms.

The next RNN step takes the second input vector and hidden state #1 to create the output of that time step. In the following visualization, each pulse for the encoder or decoder is that RNN processing its inputs and generating an output for that time step. Since the encoder and decoder are both RNNs, each time step one of the RNNs does some processing, it updates its hidden state based on its inputs and previous inputs it has seen. The decoder also maintains a hidden states that it passes from one time step to the next.

![Jay Alammar's Blog](http://jalammar.github.io/images/seq2seq_6.mp4)

[📖 READ Sequence to Sequence Learning with Neural Networks-Ilya Sutskever,Oriol Vinyals,Quoc V. Le](https://arxiv.org/pdf/1409.3215.pdf)

> ### Source: [Tutorial on NMT on github](https://raw.githubusercontent.com/tensorflow/nmt), [Jay Alammar's Blog](http://jalammar.github.io) & [Lena Voita's Blog](https://lena-voita.github.io/resources)

### Seq2Seq (LSTM) Model without Pre-trained Embeddings

In [ ]:
# seq2seq encoder-decoder

maxlen = 131
max_features = 50000
embed_size = 131

encoder_inp   = Input(shape=(maxlen,))
encoder_embed = Embedding(max_features,embed_size,input_length=maxlen,trainable=True)(encoder_inp)
encoder_lstm_cell = LSTM(60,return_state='True')
encoder_output,encoder_state_h,encoder_state_c = encoder_lstm_cell(encoder_embed)
#Creating LSTM decoder model and feeding the output states (h,c) of lstm of encoders
decoder_inp   = Input(shape=(maxlen,))
decoder_embed = Embedding(max_features,embed_size,input_length=maxlen,trainable=True)(decoder_inp)
decoder_lstm_cell = LSTM(60,return_sequences='True',return_state=True)
decoder_output,decoder_state_h,decoder_state_c = decoder_lstm_cell(decoder_embed,initial_state=[encoder_state_h,encoder_state_c])
decoder_dense_cell1 = Dense(16,activation='relu')
decoder_d_output    = decoder_dense_cell1(decoder_output)
decoder_dense_cell2 = Dense(1,activation='sigmoid')
decoder_output = decoder_dense_cell2(decoder_d_output)
model = Model([encoder_inp,decoder_inp],decoder_output) 
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
%%time
from tqdm import tqdm
from tqdm.keras import TqdmCallback

history = model.fit([X_train,X_train],y_train,batch_size=1024,epochs=2, callbacks=[TqdmCallback(verbose=0)])

In [ ]:
plot_history(history)

In [ ]:
from keras.utils import plot_model

plot_model(model,to_file="seq2seq_encoder_decoder_model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96)

In [ ]:
gc.collect()

### Seq2Seq (BiLSTM) Model with fasttext Embeddings

In [ ]:
%%time

# Using the fasttext word embeddding from crawl

fasttext_file= "../input/pretrained/crawl-300d-2M.vec"
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_file, binary=False)

word_index = tokenizer.word_index
num_tokens = len(tokenizer.word_index)+2
print("Found %s word vectors." % len(fasttext_model.vocab))

In [ ]:
## prepare a corresponding embedding matrix that used in a Keras Embedding layer. 
## It's a simple NumPy matrix where entry at index i is the pre-trained vector for the word of index i in our vectorizer's vocabulary.

embed_size = 300
embedding_matrix = np.zeros((num_tokens, embed_size))

hits = 0
misses = 0

# embedding matrix
for word, i in word_index.items():
    try:
        embedding_vector = fasttext_model.get_vector(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
    except:
        misses += 1
    
        
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
embedding_matrix.shape

In [ ]:
num_tokens = len(tokenizer.word_index)+2
num_tokens

In [ ]:
embedding_dim=300
num_tokens = len(tokenizer.word_index)+2

encoder_inp=Input(shape=(maxlen,))
encoder_embed = Embedding(num_tokens,embedding_dim,weights=[embedding_matrix],trainable=False)(encoder_inp)
encoder_lstm_cell=Bidirectional(LSTM(60,return_state='True'))
encoder_outputs,encoder_state_flstm_h,encoder_state_flstm_c,encoder_state_blstm_h,encoder_state_blstm_c=encoder_lstm_cell(encoder_embed)
encoded_states=[encoder_state_flstm_h,encoder_state_flstm_c,encoder_state_blstm_h,encoder_state_blstm_c]

# Creating LSTM decoder model and feeding the output states (h,c) of lstm of encoders
decoder_inp=Input(shape=(maxlen,))
decoder_embed=Embedding(num_tokens,embedding_dim,weights=[embedding_matrix])(decoder_inp)
decoder_lstm_cell=Bidirectional(LSTM(60,return_sequences='True',return_state=True),merge_mode="concat")
decoder_outputs,decoder_state_lstm_h,decoder_state_lstm_c,_,_=decoder_lstm_cell(decoder_embed,initial_state=[encoder_state_flstm_h,encoder_state_flstm_c,encoder_state_blstm_h,encoder_state_blstm_c])

# decoderoutputs,_,_=decoder_lstm_cell(decoder_embed,initial_state=encoded_states)


decoder_dense_cell = Dense(100,activation='relu')
decoder_d_output = decoder_dense_cell(decoder_outputs)
decoder_dense_cell2 =Dense(1,activation='sigmoid')
decoder_output =decoder_dense_cell2(decoder_d_output)
    
model = Model([encoder_inp,decoder_inp],decoder_output)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
from keras.utils import plot_model

plot_model(model,
           to_file="seq2seq_encoder_decoder_model_fast_bilstm.png",
           show_shapes=True,
           show_layer_names=True,
           rankdir="TB",
           expand_nested=False,
           dpi=96)

In [ ]:
from tqdm import tqdm
from tqdm.keras import TqdmCallback

#history = model.fit([X_train,X_train],y_train,batch_size=1024,epochs=2, verbose=0, callbacks=[TqdmCallback(verbose=0)])

## Dynamic Embeddings - ELMo (Embeddings from Language Models)

<img src="https://images.squarespace-cdn.com/content/v1/5208f2f8e4b0f3bf53b73293/1407293570153-DZJEI1WX3J5I2YK7CYAX/ke17ZwdGBToddI8pDm48kHxZW8G9yQIB8WDax7xpbxxZw-zPPgdn4jUwVcJE1ZvWhcwhEtWJXoshNdA9f1qD7WhFDJnCzrBmtNAj8M71q_A25spwTDnl_gCvSDHYIQArOx3xS_zC7rpYg0txtq7IHQ/Elmo_Blink.gif?format=2500w">


Deep contextual embeddings and sentence/word vectors falls under dynamic embeddings. These embeddings are current SOTA implying that there is a need for robust Neural Network models.

[📖 READ **Deep contextualized word representations** - Matthew E. Peters, Mark Neumann, Mohit Iyyer, Matt Gardner, Christopher Clark, Kenton Lee, Luke Zettlemoyer](https://arxiv.org/abs/1802.05365)

Both these papers are essentially important for their contributions to contextual deep embeddings.

I  highly recommend to read these papers!!! These are really cool explanation of how ELMo was designed.

ELMo deep contextualized word embeddings are helpful in achieving state-of-the-art (SOTA) results in several NLP tasks. 

![Lena Voita's Blog](https://lena-voita.github.io/resources/lectures/transfer/elmo/training-min.png)

### Under the hood:

The architecture above uses a character-level convolutional neural network (CNN) to represent words of a text string into raw word vectors
These raw word vectors act as inputs to the first layer of biLM
The forward pass contains information about a certain word and the context (other words) before that word
The backward pass contains information about the word and the context after it
This pair of information, from the forward and backward pass, forms the intermediate word vectors
These intermediate word vectors are fed into the next layer of biLM
The final representation (ELMo) is the weighted sum of the raw word vectors and the 2 intermediate word vectors

As the input to the biLM is computed from characters rather than words, it captures the inner structure of the word. For example, the biLM will be able to figure out that terms like beauty and beautiful are related at some level without even looking at the context they often appear in. Sounds incredible!


[📖 READ Analytical Vidya](https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/)

<img src ="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/03/output_YyJc8E.gif">

In [ ]:
# ELMO works well with TF 1.15 (rather any TF version <2.0.0) . For using ELMO from Tensorflow Hub, we have to follow the steps:
# Restart the Kernel
# Run the following

#!pip install --upgrade pip
#!pip install -U tensorflow==1.15
#import tensorflow as tf
#tf.__version__

In [ ]:
#elmo_embed = hub.Module("https://tfhub.dev/google/elmo/2",trainable=True)

#Creating the elmo embeddings by squeezing the inputs
#def create_embedding(z):return elmo_embed(tf.squeeze(tf.cast(z,tf.string)),signature='default',as_dict=True)["default"]

# Attention in Sequence-to-Sequence Model

> ### The Problem of Fixed Encoder Representation

The encoder of seq2seq model compressed the whole 'source sentence' into a single vector (last hidden state) as the context vector for the decoder. When the encoder is forced to put all information into a single vector, it is likely to forget something. Hence, the context vector turned out to be a bottleneck for these types of standard seq2seq models.

The encoder (RNN/LSTM) of the seq2seq model is generally unable to accurately process long input sequences/ large dependencies.

On the other hand, the Attention Mechanism directly addresses this issue as it retains and utilises 'all the hidden states of the input(source) sequence' during the decoding process.

It creates a unique mapping between the decoder output to all the encoder hidden states at each time step. For each output that the decoder makes, it has access to the entire input sequence and can selectively pick out specific elements from that sequence to produce the output.

[📖 READ Floydhub Blog ](https://blog.floydhub.com/attention-mechanism/#luong-att)

Let's explore:
- Bahdanau Attention
- Luong Attention

# Attention Mechanisms

> ### Calculating Attention Vector

![Decoder](https://jalammar.github.io/images/attention_process.mp4)

In order to focus on the parts of the input that are relevant to the decoding time step, the decoder does the following:

1. Look at the set of encoder hidden states it received – each encoder hidden states is most associated with a certain word in the input sentence
2. Give each hidden states a (alignment) score (either dot product or concatanation of encoder hidden state and previous decoder hidden state)
![](https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/g3doc/img/attention_equation_1.jpg)
3. Multiply each hidden states by its softmaxed score, thus amplifying hidden states with high scores, and drowning out hidden states with low scores.
4. Weighted sum of the vector to get Context vector for decoder for this time step.


Source: https://jalammar.github.io

## Attention Architectures

![Attention Mechanisms](https://blog.floydhub.com/content/images/2019/09/Slide41-1.JPG)
Source: Floydhub Blog

## Visualising NMT - seq2seq2 model with attention mechanism
![](https://jalammar.github.io/images/attention_tensor_dance.mp4)

Source: Jay Alammar's Blog

## Calculating the Attention (weights) in Bahdanau Attention Mechanism

![Bahdanau](https://blog.floydhub.com/content/images/2019/09/Slide50.JPG)

### Steps to calculate Bahdanau Attention Weights:
​
1. Producing the Encoder Hidden States - Encoder produces hidden states of each element in the input(source) sequence
​
2. Calculating Alignment Scores between the previous decoder hidden state and each of the encoder’s hidden states are calculated (Note: The last encoder hidden state can be used as the first hidden state in the decoder)
​
3. Softmaxing the Alignment Scores - the alignment scores for each encoder hidden state are combined and represented in a single vector and subsequently softmaxed
​
4. Calculating the Context(Attention) Vector - the encoder hidden states and their respective alignment scores are multiplied to form the context vector
​
5. Decoding the Output - the context vector is concatenated with the previous decoder output and fed into the Decoder RNN for that time step along with the previous decoder hidden state to produce a new output


The process (steps 2-5) repeats itself for each time step of the decoder until an token is produced or output is past the specified maximum length


> ### The alignment scores for Bahdanau Attention are calculated using the hidden state produced by the decoder in the previous time step and the encoder outputs.

## Calculating Attention Weights in Luong Attention Mechanism

![](https://blog.floydhub.com/content/images/2019/09/Slide51.JPG)

Source: Floydhub's Blog (www.blog.floydhub.com)

### Steps to calculate Luong Attention Weights:

1. Producing the Encoder Hidden States - Encoder produces hidden states of each element in the input sequence
2. Decoder RNN - the previous decoder hidden state and decoder output is passed through the Decoder RNN to generate a new hidden state for that time step
3. Calculating Alignment Scores - using the new decoder hidden state and the encoder hidden states, alignment scores are calculated
4. Softmaxing the Alignment Scores - the alignment scores for each encoder hidden state are combined and represented in a single vector and subsequently softmaxed
5. Calculating the Context Vector - the encoder hidden states and their respective alignment scores are multiplied to form the context vector
6. Producing the Final Output - the context vector is concatenated with the decoder hidden state generated in step 2 as passed through a fully connected layer to produce a new output

The process (steps 2-6) repeats itself for each time step of the decoder until an token is produced or output is past the specified maximum length

# Code: Sequence-to-Sequence NMT with Attention Mechanisms

> source: https://github.com/tensorflow/addons/blob/master/docs/tutorials/networks_seq2seq_nmt.ipynb

#### Dataset provided by http://www.manythings.org/anki/
​
This dataset contains language translation pairs in the format:
​
May I borrow this book?    ¿Puedo tomar prestado este libro?
​
#### After downloading the dataset, here are the steps to prepare the data
1. Add a start and end token to each sentence
2. Clean the sentences by removing special characters
3. Create a Vocabulary with word index (mapping from word → id) and reverse word index (mapping from id → word)
4. Pad each sentence to a maximum length. (Why? you need to fix the maximum length for the inputs to recurrent encoders)

In [ ]:
!pip install tensorflow-addons==0.11.2 -q
import tensorflow_addons as tfa

In [ ]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer

In [ ]:
import os
import unicodedata
import re
import io
import time

def download_nmt():
    path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

    path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
    return path_to_file

path_to_file = download_nmt()

print(path_to_file)

#### Define a NMTDataset class with necessary functions to follow Step 1 to Step 4.

The call() will return:

> train_dataset and val_dataset : tf.data.Dataset objects

> inp_lang_tokenizer and targ_lang_tokenizer : tf.keras.preprocessing.text.Tokenizer objects

In [ ]:
class NMTDataset:
    def __init__(self, problem_type='en-spa'):
        self.problem_type = 'en-spa'
        self.inp_lang_tokenizer = None
        self.targ_lang_tokenizer = None
    

    def unicode_to_ascii(self, s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    ## Step 1 and Step 2 
    def preprocess_sentence(self, w):
        w = self.unicode_to_ascii(w.lower().strip())

        # creating a space between a word and the punctuation following it
        # eg: "he is a boy." => "he is a boy ."
        # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
        w = re.sub(r"([?.!,¿])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)

        # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

        w = w.strip()

        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        w = '<start> ' + w + ' <end>'
        return w
    
    def create_dataset(self, path, num_examples):
        # path : path to spa-eng.txt file
        # num_examples : Limit the total number of training example for faster training (set num_examples = len(lines) to use full data)
        lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
        word_pairs = [[self.preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

        return zip(*word_pairs)

    # Step 3 and Step 4
    def tokenize(self, lang):
        # lang = list of sentences in a language
        
        # print(len(lang), "example sentence: {}".format(lang[0]))
        lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
        lang_tokenizer.fit_on_texts(lang)

        ## tf.keras.preprocessing.text.Tokenizer.texts_to_sequences converts string (w1, w2, w3, ......, wn) 
        ## to a list of correspoding integer ids of words (id_w1, id_w2, id_w3, ...., id_wn)
        tensor = lang_tokenizer.texts_to_sequences(lang) 

        ## tf.keras.preprocessing.sequence.pad_sequences takes argument a list of integer id sequences 
        ## and pads the sequences to match the longest sequences in the given input
        tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

        return tensor, lang_tokenizer

    def load_dataset(self, path, num_examples=None):
        # creating cleaned input, output pairs
        targ_lang, inp_lang = self.create_dataset(path, num_examples)

        input_tensor, inp_lang_tokenizer = self.tokenize(inp_lang)
        target_tensor, targ_lang_tokenizer = self.tokenize(targ_lang)

        return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

    def call(self, num_examples, BUFFER_SIZE, BATCH_SIZE):
        file_path = download_nmt()
        input_tensor, target_tensor, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(file_path, num_examples)
        
        input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

        train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

        val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
        val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

        return train_dataset, val_dataset, self.inp_lang_tokenizer, self.targ_lang_tokenizer

In [ ]:
BUFFER_SIZE = 32000
BATCH_SIZE = 32
# Let's limit the #training examples for faster training
num_examples = 18000

dataset_creator = NMTDataset('en-spa')
train_dataset, val_dataset, inp_lang, targ_lang = dataset_creator.call(num_examples, BUFFER_SIZE, BATCH_SIZE)

In [ ]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

### Parameters

In [ ]:
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
max_length_input = example_input_batch.shape[1]  # max_length_english
max_length_output = example_target_batch.shape[1]# max_length_spanish

embedding_dim = 256
units = 1024
steps_per_epoch = num_examples//BATCH_SIZE


print("max_length_english, max_length_spanish; vocab_size_english, vocab_size_spanish")
max_length_input, max_length_output,vocab_inp_size, vocab_tar_size

## Encoder Class

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
          ##-------- LSTM layer in Encoder ------- ##
        self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,return_sequences=True,
                                               return_state=True,
                                               recurrent_initializer='glorot_uniform')
    def call(self, x, hidden):
        x = self.embedding(x)
        output, h, c = self.lstm_layer(x, initial_state = hidden)
        return output, h, c
    
    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_size, self.enc_units)), tf.zeros((self.batch_size, self.enc_units))] # all zeros

In [ ]:
## Test Encoder Stack

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

In [ ]:
sample_hidden

## Decoder Class

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size, attention_type='luong'):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.attention_type = attention_type
        #Final Dense layer on which softmax will be applied
        self.fc = tf.keras.layers.Dense(vocab_size)
        # Define the fundamental cell for decoder recurrent structure
        self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)
        
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        
        # Create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(self.dec_units,None,
                                                                  self.batch_size*[max_length_input], self.attention_type)
        # Wrap attention mechanism with the fundamental rnn cell of decoder
        self.rnn_cell = self.build_rnn_cell(batch_size)
        # Define the decoder with respect to fundamental rnn cell
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)
    
    def build_rnn_cell(self, batch_size):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell,self.attention_mechanism,
                                                attention_layer_size=self.dec_units)
        return rnn_cell

    def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # type: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)
        if(attention_type=='bahdanau'):
            return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
        else:
            return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

    def build_initial_state(self, batch_size, encoder_state, Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size, dtype=Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
        return decoder_initial_state
    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_size*[maxlen_output-1])
        return outputs

In [ ]:
# Test decoder stack 

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'bahdanau')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)

## Define Optimizer and Loss function

In [ ]:
optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
  # real shape = (BATCH_SIZE, max_length_output)
  # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
    cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = cross_entropy(y_true=real, y_pred=pred)
    mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)  
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [ ]:
#Checkpoints 

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## One train_step Operation

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


    dec_input = targ[ : , :-1 ] # Ignore <end> token
    real = targ[ : , 1: ]         # ignore <start> token

    # Set the AttentionMechanism object with encoder_outputs
    decoder.attention_mechanism.setup_memory(enc_output)

    # Create AttentionWrapperState as initial_state for decoder
    decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
    pred = decoder(dec_input, decoder_initial_state)
    logits = pred.rnn_output
    loss = loss_function(real, logits)
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return loss

In [ ]:
gc.collect()

In [ ]:
%%time

EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    # print(enc_hidden[0].shape, enc_hidden[1].shape)
    
    for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch,batch_loss.numpy()))
        # saving (checkpoint) the model every 2 epochs
if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

## Using tf-addons BasicDecoder for decoding¶

In [ ]:
def evaluate_sentence(sentence):
    sentence = dataset_creator.preprocess_sentence(sentence)
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],maxlen=max_length_input,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''
    
    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)
    dec_h = enc_h
    dec_c = enc_c
    
    start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<start>'])
    end_token = targ_lang.word_index['<end>']

    greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()
    
    # Instantiate BasicDecoder object
    decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, 
                                                sampler=greedy_sampler, output_layer=decoder.fc)
    # Setup Memory in decoder stack
    decoder.attention_mechanism.setup_memory(enc_out)
    # set decoder_initial_state
    decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)


  ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
  ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
  ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function

    decoder_embedding_matrix = decoder.embedding.variables[0]
    outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, 
                                     end_token= end_token, initial_state=decoder_initial_state)
    return outputs.sample_id.numpy()

def translate(sentence):
    result = evaluate_sentence(sentence)
    print(result)
    result = targ_lang.sequences_to_texts(result)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(u'hace mucho frio aqui.')

In [ ]:
translate(u'¿todavia estan en casa?')

# Transformer Architecture Explained!!

[📖 READ **Attention Is All You Need**- Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhine](https://arxiv.org/abs/1706.03762)

### Self-Attention (Scaled Dot-Product Attention)

Transformer is a model introduced in the paper Attention is All You Need in 2017. It is based solely on attention mechanisms.

Currently, Transformers (with variations) are de-facto standard models not only in sequence to sequence tasks but also for language modeling and in pretraining settings.

#### Why self-attention?

Transformer's encoder tokens interact with each other all at once and exchanges information to understand each other better in the context of the whole sentence. This happens in several layers (N=6). Basically, self-attention layer computes similarity scores among the tokens, and normalised to give the better representation of input/output tokens.

In each decoder layer, tokens of the prefix also interact with each other via a self-attention mechanism, but additionally, they look at the encoder states.

The difference between "attention" and "self-attention" is that self-attention operates between representations of the same nature: e.g., all encoder states in some **K**th layer.

#### Calculating self-attention weights?

**The first step** in calculating self-attention matrix is to create three vectors from each of the encoder’s input vectors (e.g.,X). So for each word, we create a Query vector(Q), a Key vector(K), and a Value vector(V). 

![fig](https://jalammar.github.io/images/t/self-attention-matrix-calculation.png)

In the above figure, Wq, Wk,Wv are weight vectors are of dimensionality [maximum sequence length x 64] to make the computations faster. These vectors are randomly initialised and learn the accurate weights while training.

#### Query, Key, and Value in Self-Attention

Each input token in self-attention receives three representations corresponding to the roles it can play:

query (Q) - asks for information (how similar is the token vis-a-vis other tokens?)

key (K)   - says it has some information

value (V) - gives the information

The query is used when a token looks at others - it's seeking the information to understand itself better. 
The key is responding to a query's request: it is used to compute attention weights. 
The value is used to compute attention output: it gives information to the tokens which "say" they need it (i.e. assigned large weights to the similar tokens).

1. compute the dot product between query matrix and key matrix, QK^T
2. divide QK^T by square-root of Key vector
3. apply the softmax to normalise the scores to obtain the score matrix
4. compute the attention weights matrix by multiplying the score matrix by the value matrix V.

![self attention score](https://www.tensorflow.org/images/tutorials/transformer/scaled_attention.png)


![fig](https://images.prismic.io/peltarionv2/a7445eec-3dd8-4e62-948c-e3c3f9b3a987_self-attention_head.svg?auto=compress%2Cformat&rect=0%2C0%2C148%2C150&w=1980&h=2007)

### The following clip illustrates the self-attention:

![](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/encoder_self_attention.mp4)

### Multi-head Attention

If the Value vector of other tokens dominates the actual token, where the actual token is ambigous, then the dominance is useful. Otherwise, it may cause misunderstanding the right representation of the token. So, instead of computing single attention, we compute multiple attention weights over respective multiple heads and concatanate the final vectors.

Thus, multi-head attention will be more accurate.

![fig](https://jalammar.github.io/images/t/transformer_attention_heads_weight_matrix_o.png)

![](http://www.tensorflow.org/images/tutorials/transformer/multi_head_attention.png)

### Figure: Multi-head Attention

![](https://images.prismic.io/peltarionv2/ee0b5ada-211f-4d33-ba6b-91a2ace50714_self-attention_multihead.svg?auto=compress%2Cformat&rect=0%2C0%2C133%2C150&w=1980&h=2233)

Source: [Lena Voita's Blog](https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html#transformer_intro); [Jay Allamar's Blog](https://jalammar.github.io/illustrated-transformer); www.peltarion.com ;www.tensorflow.org

### Masked Self-Attention of Decoder

In the Transdormer decoder, self-attention is a bit different from the one in the encoder. 

The encoder receives all tokens at once, while the decoder generates one token at a time: during generation, it doesn't know which tokens will be generated in the future.

To forbid the decoder to look ahead, the model uses masked self-attention: future tokens are masked out. 

![](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/masked_self_attn.mp4)

#### Learn the position with Positional Encoding

To account for the order of the words/tokens in the input sequence, the transformer adds a positional encoding to each input embedding token.These vectors follow a specific pattern that the model learns, which helps it determine the position of each word, or the distance between different words in the sequence. 

![](https://miro.medium.com/max/1572/1*xCeAOFp17t-NcWWpF2k9Gw.png)

#### Residuals

Each layer has a feed-forward network block: two linear layers with ReLU non-linearity between them:
and is followed by a layer-normalization step.ach layer has a feed-forward network block: two linear layers with ReLU non-linearity between them:

![](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/ffn-min.png)


<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>Feed</mi>
  <mi>Forward</mi>
  <mi>Network</mi>
  <mo stretchy="false">(</mo>
  <mi>x</mi>
  <mo stretchy="false">)</mo>
  <mo>=</mo>
  <mo data-mjx-texclass="OP" movablelimits="true">max</mo>
  <mo stretchy="false">(</mo>
  <mn>0</mn>
  <mo>,</mo>
  <mi>x</mi>
  <msub>
    <mi>W</mi>
    <mn>1</mn>
  </msub>
  <mo>+</mo>
  <msub>
    <mi>b</mi>
    <mn>1</mn>
  </msub>
  <mo stretchy="false">)</mo>
  <msub>
    <mi>W</mi>
    <mn>2</mn>
  </msub>
  <mo>+</mo>
  <msub>
    <mi>b</mi>
    <mn>2</mn>
  </msub>
  <mo>.</mo>
</math>

##  Layer Normalization

![](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/layer_norm-min.png)

It independently normalizes vector representation of each example in batch - this is done to control "flow" to the next layer. Layer normalization improves convergence stability and sometimes even quality.

**LayerNorm** has trainable parameters, 
scale
and 
bias,
which are used after normalization to rescale layer's outputs (or the next layer's inputs). Note that 
μ
k
 and 
σ
k
are evaluated for each example, but 
scale
and 
bias,
are the same - these are layer parameters.

## Jotting all the components - Transformer Architecture

![fig](https://www.tensorflow.org/images/tutorials/transformer/transformer.png)

The final linear layer is a fully connected neural network that projects the vector produced by the stack of decoders, into a much, much larger vector called a logits vector. The softmax layer then turns those scores into probabilities (all positive, all add up to 1.0). The cell with the highest probability is chosen, and the word associated with it is produced as the output for this time step.

![predict](https://jalammar.github.io/images/t/transformer_decoder_output_softmax.png)

## Loss function

As we know, we compare two probability distributions with Cross-entropy /KL Divergence. Cross entropy is loss function and choose Adam as optimizer.

The model produces the outputs one at a time, we can assume that the model is selecting the word with the highest probability from that probability distribution and throwing away the rest. That’s one way to do it (called greedy decoding).

In [ ]:
#!pip install pyyaml~=3.12
!pip install nbdime==2.0.0

In [ ]:
!python3.7 -m pip install --upgrade pip
!pip install --upgrade requests -q

## Installing Transformers library

In [ ]:
!pip install transformers -q

In [ ]:
from transformers import pipeline
print(pipeline('sentiment-analysis')('we do not love you'))

**Please visit the kernel [Text Classification with BERT](https://www.kaggle.com/rizdelhi/text-classification-with-bert)to understand about the Transformer Code and BERT.**